<a href="https://colab.research.google.com/github/obeabi/SdsOctober2020Challenge/blob/main/FlipKart_Reviws_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web scraper for text review
## Written by Abiola Obembe
### Date: 2021-01-12
### Course: ML masters with deployment and deep learning

In [3]:
# Importing necessary libraries

from flask import Flask, render_template, request,jsonify
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pymongo

print("Libraries installed successfully!")

Libraries installed successfully!


In [4]:
# Create Flask object
app = Flask(__name__)  # initialising the flask app with the name 'app'


## Build Main Body of Code

In [64]:
# Type search string and build complete url
searchString = 'samsung'
searchString = searchString.replace(" ", "")
flipkart_url = "https://www.flipkart.com/search?q=" + searchString # preparing the URL to search the product on flipkart

print(flipkart_url)

https://www.flipkart.com/search?q=samsung


In [115]:
# Request webpage from the internet
#uClient = requests.get(flipkart_url) # requesting the webpage from the internet
uClient = uReq(flipkart_url)
flipkartPage = uClient.read() # reading the webpage
uClient.close() # closing the connection to the web server
uClient

In [166]:
# Clean the data to be scraped
# Parse the webapage as HTML and find bigboxes that contains each device listed
flipkart_html = bs(flipkartPage, "html.parser") # parsing the webpage as HTML
total_boxes = []
bigboxes = flipkart_html.findAll("div", { "class": "_2pi5LC col-12-12"})
del bigboxes[0:3] 
del bigboxes[-4:] 
entries = len(bigboxes)
bigboxes[-5:]
  

[<div class="_2pi5LC col-12-12"><div class="_13oc-S"><div data-id="MOBFRZZHHUZTTYCZ" style="width:100%"><div class="_2kHMtA"><a class="_1fQZEK" href="/samsung-galaxy-m11-metallic-blue-32-gb/p/itme875f568bb05b?pid=MOBFRZZHHUZTTYCZ&amp;lid=LSTMOBFRZZHHUZTTYCZGZVDV8&amp;marketplace=FLIPKART&amp;srno=s_1_20&amp;otracker=search&amp;fm=organic&amp;iid=515a47a5-de33-4dad-953d-d71b7397b585.MOBFRZZHHUZTTYCZ.SEARCH&amp;ssid=yxgewpv5ts0000001610480875979&amp;qH=fe546279a62683de" rel="noopener noreferrer" target="_blank"><div class="MIXNux"><div class="_2QcLo-"><div><div class="CXW8mj" style="height:200px;width:200px"><img alt="Samsung Galaxy M11 (Metallic Blue, 32 GB)" class="_396cs4" src="//img1a.flixcart.com/www/linchpin/fk-cp-zion/img/placeholder_fcebae.svg"/></div></div></div><div class="_3wLduG"><div class="_3PzNI-"><span class="f3A4_V"><div class="_2iDkf8"><label><input class="_30VH1S" readonly="" type="checkbox"/><div class="_24_Dny"></div></label></div></span><label class="_6Up2sF"><span>

In [171]:
# Scrape data from big boxes and Iterate through the boxes
entries = len(bigboxes)
#table = db[searchString] # creating a collection with the same name as search string. Tables and Collections are analogous.
total_pages = 10
for page in range(total_pages+1):
  total_reviews = [] # initializing an empty list for reviews
  box = bigboxes[i] #  taking the first iteration (for demo)
  productLink = "https://www.flipkart.com" + box.div.div.div.a['href'] # extracting the actual product link
  prodRes = requests.get(productLink) # getting the product page from serve
  prod_html = bs(prodRes.text, "html.parser") # parsing the product page as HTML

  commentboxes = prod_html.find_all('div', {'class': "_16PBlm"}) # finding the HTML section containing the customer comments
  #table = db[searchString] # creating a collection with the same name as search string. Tables and Collections are analogous.
  #  iterating over the comment section to get the details of customer and their comments
  for commentbox in commentboxes:
      try:              
         name = commentbox.div.div.find_all('p', {'class': '_2sc7ZR _2V5EHH'})[0].text
      except:
         name = 'No Name'

      try:
        rating = commentbox.div.div.div.div.text

      except:
        rating = 'No Rating'

      try:
        commentHead = commentbox.div.div.div.p.text
      except:
        commentHead = 'No Comment Heading'
      try:
        comtag = commentbox.div.div.find_all('div', {'class': ''})
        custComment = comtag[0].div.text
      except:
        custComment = 'No Customer Comment'
      mydict = {"Product": searchString, "Name": name, "Rating": rating, "CommentHead": commentHead,
                              "Comment": custComment} # saving that detail to a dictionary
      #x = table.insert_one(mydict) #insertig the dictionary containing the rview comments to the collection
      total_reviews.append(mydict) #  appending the comments to the review list
  

In [170]:
# Create dataframe with appropriate headers
columns = ['Comment', 'CommentHead','Name', 'Product', 'Rating']
df = pd.DataFrame (total_reviews,columns=columns)

df.head()

,Comment,CommentHead,Name,Product,Rating
0,Awsome phone best camera best and beast batte...,Mind-blowing purchase,Ashwin Rathakaran,samsung,5
1,No regretsI'm just blown away by the budget fr...,Excellent,Saurabh Kumar,samsung,5
2,"Nice phone , camera is nice, display is super,...",Really Nice,Dragon Peps,samsung,4
3,Amazing phone. Superb camera front and back. A...,Really Nice,Dishi Mehrotra,samsung,4
4,Excellent product with best viewing experience...,Terrific,Flipkart Customer,samsung,5


In [169]:
# Let's print shape of dataframe
df.shape

(253, 5)